In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from tensorflow.keras import layers
import itertools

In [2]:
def loadParms():
    l1Size     = [18,12,6]
    l2Size     = [6,3]
    l3Size     = [4,2]
    lr         = [2e-4]
    Lambda     = [1e-4]          # Regularization parameter
    batchSize  = [50]
    epochs     = [10]
    activation = [tf.nn.tanh]           # 'tanh' 'leakyReLU' 'ReLU' 'relu6' 'elu' 'crelu'
    stdDev     = [0.2]
    
    return list(itertools.product(l1Size,
                                  l2Size,
                                  l3Size,
                                  lr,
                                  Lambda,
                                  batchSize,
                                  epochs,
                                  activation,
                                  stdDev))

In [3]:
def getData():
    folder = "/home/tbrownex/data/test/"
    file   = "autoEncoder.csv"
    df = pd.read_csv(folder+file,index_col=0)
    cols = ["v1","v2","v3","v4","v5","v6","v7","v8","v9","label"]
    df.columns = cols
    return df

In [4]:
def prepData(df):
    train, test = train_test_split(df, test_size=0.2)
    train = train[train['label'] == 0]     # Train on "normal" data only
    del train["label"]
    del test["label"]
    
    scaler = MinMaxScaler()
    train = scaler.fit_transform(train)
    test  = scaler.transform(test)
    return train, test

In [5]:
def buildLayers(parmDict):
    nn = tf.keras.Sequential()
    nn.add(layers.Dense(parmDict["l1Size"], activation=parmDict["activation"]))
    nn.add(layers.Dense(parmDict["l2Size"], activation=parmDict["activation"]))
    nn.add(layers.Dense(parmDict["l3Size"], activation=parmDict["activation"]))
    nn.add(layers.Dense(parmDict["l2Size"], activation=parmDict["activation"]))
    nn.add(layers.Dense(parmDict["l1Size"], activation=parmDict["activation"]))
    nn.add(layers.Dense(9, activation=parmDict["activation"]))
    return nn

In [6]:
def buildNetwork(parmDict):
    nn = buildLayers(parmDict)
    nn.compile(optimizer=tf.train.AdamOptimizer(parmDict["lr"]),\
               loss='mse',\
               metrics=['accuracy'])
    return nn

In [7]:
def fitNetwork(nn):
    return nn.fit(trainData, trainData,\
                  batch_size=32,\
                  epochs=40,\
                  validation_split=0.15,\
                  verbose=0,\
                  shuffle=False)

In [8]:
def evaluateNetwork(nn):
    return nn.evaluate(x=testData,\
                       y=testData,\
                       verbose=0)

In [9]:
df = getData()
trainData, testData = prepData(df)

parms = loadParms()

/home/tbrownex/tensorflow/lib/python3.5/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [10]:
print("{:<30}{:<8}{}".format("parms","mse","acc"))

for x in parms:
    parmDict = {}                  # holds the hyperparameter combination for one run
    parmDict['l1Size']     = x[0]
    parmDict['l2Size']     = x[1]
    parmDict['l3Size']     = x[2]
    parmDict['lr']         = x[3]
    parmDict['lambda']     = x[4]
    parmDict['batchSize']  = x[5]
    parmDict['epochs']     = x[6]
    parmDict['activation'] = x[7]
    parmDict['stdDev']     = x[8]
    
    nn = buildNetwork(parmDict)
    _ = fitNetwork(nn)
    mse, acc = evaluateNetwork(nn)
    print(x)
    print("{:<10.3f}{:.3f}".format(mse, acc))

parms                         mse     acc
(18, 6, 4, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.008     0.989
(18, 6, 2, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.017     0.969
(18, 3, 4, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.017     0.969
(18, 3, 2, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.021     0.969
(12, 6, 4, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.016     0.969
(12, 6, 2, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.017     0.969
(12, 3, 4, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.016     0.969
(12, 3, 2, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.015     0.982
(6, 6, 4, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.026     0.969
(6, 6, 2, 0.0002, 0.0001, 50, 10, <function tanh at 0x7f2bc09ea950>, 0.2)
0.021     0.969
(6, 3, 4, 0.0002, 0.0001, 50, 10, <function tanh a